In [ ]:
!pip install google-cloud-aiplatform


In [ ]:
REGION = "us-central1"

In [ ]:
import vertexai

In [ ]:
vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

In [ ]:
from google.cloud import bigquery

In [ ]:
bq_client = bigquery.Client(project=PROJECT_ID,
                            credentials = credentials)

In [ ]:
QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

In [ ]:
query_job = bq_client.query(QUERY_TABLES)

In [ ]:
for row in query_job:
    for value in row.values():
        print(value)

In [ ]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

In [ ]:
import pandas as pd

In [ ]:
query_job = bq_client.query(INSPECT_QUERY)

In [ ]:
stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
stack_overflow_df.head()

In [ ]:
QUERY_ALL = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
"""

In [ ]:
query_job = bq_client.query(QUERY_ALL)

In [ ]:
try:
    stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
except Exception as e:
    print('The DataFrame is too large to load into memory.', e)

In [ ]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

In [ ]:
query_job = bq_client.query(QUERY)

In [ ]:
### this may take some seconds to run
stack_overflow_df = query_job.result()\
                        .to_arrow()\
                        .to_pandas()

stack_overflow_df.head(2)

In [ ]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

In [ ]:
stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' '\
    + stack_overflow_df['input_text']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, evaluation = train_test_split(
    stack_overflow_df,
    ### test_size=0.2 means 20% for evaluation
    ### which then makes train set to be of 80%
    test_size=0.2,
    random_state=42
)

In [ ]:
import datetime

In [ ]:
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")

In [ ]:
cols = ['input_text_instruct','output_text']
tune_jsonl = train[cols].to_json(orient="records", lines=True)

In [ ]:
training_data_filename = f"tune_data_stack_overflow_\
                            python_qa-{date}.jsonl"

In [ ]:
with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)